In [2]:
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [3]:
folder = './aclImdb'

In [4]:
labels = {'pos': 1, 'neg': 0}

In [5]:
df = pd.DataFrame()

In [6]:
for f in ('test', 'train'):    
    for l in ('pos', 'neg'):
        path = os.path.join(folder, f, l)
        for file in os.listdir (path) :
            with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]],ignore_index=True)

df.columns = ['review', 'sentiment']

In [7]:
df.to_csv('movie_data.csv',index=False,encoding='utf-8')

In [8]:
df.head()

,review,sentiment
0,"This movie travels farther on 8 gunshots, 2 ki...",1
1,... but I laughed. A lot.<br /><br />I saw 'As...,1
2,I am appalled and dismayed that the Network ha...,1
3,"""Throw Momma From the Train"" is a simple dark ...",1
4,I really love anything done by Savage Steve Ho...,1


In [10]:
import nltk
# nltk.download('punkt')

reviews = df.review.str.cat(sep=' ')

#function to split text into word
tokens = word_tokenize(reviews)

vocabulary = set(tokens)
print(len(vocabulary))

#frequency_dist = nltk.FreqDist(tokens)
#sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]

[nltk_data] Downloading package punkt to /home/freakcap/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


198763


In [12]:
# nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
tokens = [w for w in tokens if not w in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/freakcap/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
#Stemming
stemmer=PorterStemmer()
tokens=[stemmer.stem(word) for word in tokens]

In [14]:
#Dividing dataset into test and train

X_train = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [15]:
#Using TFIDF to convert text corpus to feature vectors

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

(25000, 73822) (25000, 73822)


In [16]:
#Import svm model
from sklearn import svm


In [17]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(train_vectors, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [18]:
from  sklearn.metrics  import accuracy_score
from sklearn import metrics

predicted = clf.predict(test_vectors)

print("Accuracy:",accuracy_score(y_test,predicted))
print("Precision:",metrics.precision_score(y_test, predicted))
print("Recall:",metrics.recall_score(y_test, predicted))

Accuracy: 0.88064
Precision: 0.8880913539967373
Recall: 0.87104
